# Computer Lab: Nonnegative Matrix Factorization
## 1、 Database

In [1]:
import matplotlib.pyplot as plt
import os
import numpy as np
import time
im=plt.imread(".\\orl_faces\\s1\\1.pgm")
plt.imshow(im)

In [2]:
print("The sieze of image is:",im.shape)
im_h=np.ravel(im)

The sieze of image is: (112, 92)


There are 400 images in the database, each imamge has 10304 pixels with size (112,92)

## 2、 Presentation of the model

![title](2.1.jpg)
![title](2.2.jpg)

    So we have proved that the objective function is not convex. Its gradient is (-(M-WH)H'/np),-W'(M-WH)/np) and the gradient doesn't satisfy Lipschitz continuous
   
## 3 Find W when H0 is fixed

### Questions 3.1
#### We get one possible solution H0 and W0 through this way. And we have choosed k bases who include the biggest singular values.  We could use the random way to initialization.

### Questions 3.2
![title](3.2.jpg)

### Question 3.3

In [3]:
M=[]
for i in range(1,11):
    im=plt.imread(".\\orl_faces\\s1\\"+str(i)+".pgm")
    M.append(np.ravel(im))
M=np.array(M,dtype=float)
M.shape

(10, 10304)

In [4]:
from scipy.sparse.linalg import svds
k=5
[n,p]=M.shape
M=np.array(M,dtype=float)
W0, S, H0 = svds(M, k)
W0 = np.maximum(0, W0 * np.sqrt(S))
H0 = np.maximum(0,(H0.T * np.sqrt(S)).T)

In [5]:
def g_w(w,h=H0):
    [n,p]=M.shape
    w=w.reshape(n,k)
    m_t=np.dot(w,h)
    g=np.linalg.norm(m_t-M,ord='fro')**2
    g=g/(2*n*p)
    return g

In [6]:
def g_gradient(w,h=H0):
    [n,p]=M.shape
    w=w.reshape(n,k)
    m_t=np.dot(w,h)
    g=-1*np.dot((M-m_t),h.T)/(n*p)
    return np.ravel(g)

In [7]:
t=g_gradient(W0)
t.shape
from scipy.optimize import check_grad
print("The result of function scipy.optimize.check grad is:",check_grad(g_w,g_gradient,np.ravel(W0)))
print("It seems accpetable")

The result of function scipy.optimize.check grad is: 3.6764329094003976e-05
It seems accpetable


### Question 3.4

We know:
  
$$prox_{\gamma lR+}=\underset{\gamma }{argmin} (l_{R_+}(\gamma)+\frac{||\gamma-x||^2}{2})$$

To get the minimize result, the $\gamma$ must bigger than 0, otherwise the $l_{R_+}(\gamma)=+\infty$

So we get the  $\gamma\in R_+$

When we  $\gamma> 0$, the problem becomes to 

$$\underset{\gamma }{argmin} ||\gamma-x|| ,\quad  where \quad \gamma> 0  $$

It equal to: $$proj_{R_+}(x)$$

### Question 3.5

In [8]:
def projected_gradient_method(val_g,grad_g,W0,h,gamma,N):
    w=W0
    for i in range(N):
        w=w-gamma*grad_g(W0,h).reshape(n,k)
        w=np.maximum(0,w)
        W0=w
    return(val_g(w,h),w)

### Question 3.6

In [9]:
print("Before the iterations, the value of g(W0) is :",g_w(W0))
gamma=1
N=100
start=time.time()
g_new,w_new=projected_gradient_method(g_w,g_gradient,W0,H0,gamma,N)
end=time.time()
print("After N=100 times iterations with the learning rate=1,we get new value of g(w) is:",g_new)
print("The time it costs is :",end-start)

Before the iterations, the value of g(W0) is : 414.39665223126826
After N=100 times iterations with the learning rate=1,we get new value of g(w) is: 360.95642813130837
The time it costs is : 0.06699252128601074


## 4 Algorithmic renement for the problem with H0 fixed
### Question 4.1

In [10]:
def projected_gradient_method_line_search(val_g,grad_g,W0,alpha,gamma,N):
    w=W0
    for i in range(N):
        w=w-(gamma*grad_g(W0)).reshape(n,k)
        w=np.maximum(0,w)
        w=W0+alpha*(w-W0)
        W0=w
    return(val_g(w),w)

In [11]:
print("Before the iterations, the value of g(W0) is :",g_w(W0))
gamma=1
N=100
alpha=0.5
start=time.time()
g_new,w_new=projected_gradient_method_line_search(g_w,g_gradient,W0,alpha,gamma,N)
end=time.time()
print("After N=100 times iterations with the learning rate=1,alpha=0.5,we get new value of g(w) is:",g_new)
print("The time it costs is :",end-start)

Before the iterations, the value of g(W0) is : 414.39665223126826
After N=100 times iterations with the learning rate=1,alpha=0.5,we get new value of g(w) is: 363.8822461774789
The time it costs is : 0.07741212844848633


### Question 4.2
When we apply a line search,we set the $\alpha=0.5$ and the speed for iterations becomes slower. After N=100 times the g(w) becomes 363.89 compared the original methode of 360.96

## 5 Resolution of the full problem
### Question 5.1

In [12]:
def g_h(h,w=W0):
    [n,p]=M.shape
    h=h.reshape(k,p)
    m_t=np.dot(w,h)
    g=np.linalg.norm(m_t-M,ord='fro')**2
    g=g/(2*n*p)
    return g

In [13]:
def g_gradient_h(h,w=W0):
    [n,p]=M.shape
    h=h.reshape(k,p)
    m_t=np.dot(w,h)
    g=-1*np.dot(w.T,(M-m_t))/(n*p)
    return np.ravel(g)

In [14]:
def projected_gradient_method_line_search_wh(val_g,grad_g,grad_h,w,h,alpha,gamma,N):
    for i in range(N):
        W0,H0=w,h
        w=w-(gamma*grad_g(W0)).reshape(n,k)
        w=np.maximum(0,w)
        w=W0+alpha*(w-W0)
        h=h-(gamma*grad_h(H0)).reshape(k,p)
        h=np.maximum(0,h)
        h=H0+alpha*(h-H0)
    return(val_g(w,h),w,h)

In [15]:
print("Before the iterations, the value of g(W0) is :",g_w(W0))
gamma=1
N=1000
alpha=0.5
start=time.time()
g_new,w_new,h_new=projected_gradient_method_line_search_wh(g_w,g_gradient,g_gradient_h,W0,H0,alpha,gamma,N)
end=time.time()
print("After N=1000 times iterations with the learning rate=1,alpha=0.5,we get new value of g(w) is:",g_new)
print("The time it costs is :",end-start)

Before the iterations, the value of g(W0) is : 414.39665223126826
After N=1000 times iterations with the learning rate=1,alpha=0.5,we get new value of g(w) is: 304.8092383649963
The time it costs is : 3.825888156890869


### Question 5.2
For the $W_t$ and $H_t$, we get the $W_{t+1}$ by searching the minimized value for the objective value with $H_t$, so the objective value for $W_{t+1}$ and $H_t$ decreases in this process. The same reason, we could also get the smaller objective value for searching $H_{t+1}$ when $W_{t+1}$ is not changed. So the final objective value for $W_{t+1}$ and $H_{t+1}$ decreases in every iteration.

### Question 5.3


In [16]:
def projected_gradient_method_H(val_h,grad_h,H0,w,gamma,N):
    h=H0
    for i in range(N):
        h=h-gamma*grad_h(h,w).reshape(k,p)
        h=np.maximum(0,h)
        H0=h
    return(val_h(h,w),h)

In [17]:
def alternate_minimization_method(W0,H0,t):
    w,h=W0,H0
    gamma=1
    N=100
    for i in range(t):
        value,w=projected_gradient_method(g_w,g_gradient,w,h,gamma,N)
        value,h=projected_gradient_method_H(g_h,g_gradient_h,h,w,gamma,N)      
        print("After the",i+1," times iterations the value of the objective is:",value)
    return(value,w,h)    

In [19]:
t=10
start=time.time()
value,w_new,h_new=alternate_minimization_method(W0,H0,t)
end=time.time()
print("The final value of objective is",value)
print("The time it costs is :",end-start)

After the 1  times iterations the value of the objective is: 244.29925392619512
After the 2  times iterations the value of the objective is: 182.04171373838338
After the 3  times iterations the value of the objective is: 162.28036141287564
After the 4  times iterations the value of the objective is: 153.00596446942504
After the 5  times iterations the value of the objective is: 147.6585039065165
After the 6  times iterations the value of the objective is: 144.01616494608174
After the 7  times iterations the value of the objective is: 141.35705914316492
After the 8  times iterations the value of the objective is: 139.31406992573352
After the 9  times iterations the value of the objective is: 137.6779678619709
After the 10  times iterations the value of the objective is: 136.32265195437034
The final value of objective is 136.32265195437034
The time it costs is : 4.228513956069946


### Question 5.4
   The alternate minimizations method gets the better result which reduce the value of objective to 136.3, while the projected gradient mothod only reduce the value of objective to 304.8. So from the aspect of the objective value,the alternate minimizations method is better.
   Comparing the computing time,the alternate minimizations method costs about 4.2s and the projected gradient mothod costs about 3.8s. There are not big differences.

### Question 5.5

 If the $||g(W_t)-g(W_{t+1})||<\epsilon$,with the $\epsilon$ is a selected small value like 0.01, we could stop the iteration. We could use this as the stopping criterion.